In [ ]:
!pip install eagerx-tutorials  >> /tmp/eagerx_install.txt
if 'google.colab' in str(get_ipython()):
  !{"curl 'https://raw.githubusercontent.com/eager-dev/eagerx_tutorials/master/scripts/setup_colab.sh' > ~/setup_colab.sh"}
  !{"bash ~/setup_colab.sh"}

# Setup interactive notebook
from eagerx_tutorials import helper
helper.setup_notebook()

# Allows reloading of registered entites from changed files
# Required in interactive notebooks only.
%reload_ext autoreload
%autoreload 1

In [2]:
import eagerx

In [3]:
eagerx.initialize("eagerx_core", anonymous=True, log_level=eagerx.log.INFO)

... logging to /home/r2ci/.ros/log/b6fa50c2-bd58-11ec-8f0d-010fae9ecfd0/roslaunch-r2ci-Alienware-m15-R4-14318.log
started roslaunch server http://192.168.68.129:39911/
ros_comm version 1.15.14


SUMMARY

PARAMETERS
 * /rosdistro: noetic
 * /rosversion: 1.15.14

NODES

auto-starting new master
process[master]: started with pid [14368]
ROS_MASTER_URI=http://localhost:11311
setting /run_id to b6fa50c2-bd58-11ec-8f0d-010fae9ecfd0
process[rosout-1]: started with pid [14394]
started core service [/rosout]


In [7]:
from eagerx.core.env import EagerxEnv
from eagerx.core.graph import Graph
from eagerx.wrappers import Flatten

# Implementation specific
import eagerx.nodes  # Registers butterworth_filter # noqa # pylint: disable=unused-import
import eagerx_ode  # Registers OdeBridge # noqa # pylint: disable=unused-import
import eagerx_tutorials  # Registers Pendulum # noqa # pylint: disable=unused-import

In [8]:
# Other
import numpy as np
import stable_baselines3 as sb

In [9]:
# Define rate (depends on rate of ode)
rate = 30.0

# Initialize empty graph
graph = Graph.create()

# Create pendulum
pendulum = eagerx.Object.make("Pendulum", "pendulum", sensors=["pendulum_output", "action_applied"])

graph.add(pendulum)

# Create Butterworth filter
bf = eagerx.Node.make("ButterworthFilter", name="bf", rate=rate, Wn=13)
graph.add(bf)

# Connect the nodes
graph.connect(action="action", target=bf.inputs.signal)
graph.connect(source=bf.outputs.filtered, target=pendulum.actuators.pendulum_input)
graph.connect(source=pendulum.sensors.pendulum_output, observation="observation", window=1)
graph.connect(source=pendulum.sensors.action_applied, observation="action_applied", window=1)

# Define bridges
bridge = eagerx.Bridge.make("OdeBridge", rate=rate, is_reactive=True)

# Define step function
def step_fn(prev_obs, obs, action, steps):
    state = obs["observation"][0]
    u = action["action"][0]
    
    # Calculate reward
    sin_th, cos_th, thdot = state
    th = np.arctan2(sin_th, cos_th)
    
    cost = th**2 + 0.1 * thdot**2 + 0.001 * (u**2)
    
    # Determine done flag
    done = steps > 500
    
    # Set info:
    info = dict()
    
    return obs, -cost, done, info

# Initialize Environment
env = Flatten(EagerxEnv(name="rx", rate=rate, graph=graph, bridge=bridge, step_fn=step_fn))

# Initialize learner
model = sb.SAC("MlpPolicy", env, verbose=1, device="cpu")

# Train for 3 minutes
model.learn(total_timesteps=int(1800 * rate))

AssertionError: No entities of type "Object" registered.